# Creating the main_cell & JJ and JJ-Array

## Positive markers and frame by Qiskit-Metal

In [53]:
import gdspy
from importlib import reload
from itertools import product
reload(gdspy)
lib = gdspy.GdsLibrary()


#######################################################################################

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')

unground = gdspy.Rectangle((-500,-500), (500, 500), layer=2)
main_cell.add(unground)

      
#logo and the device name
#device_name = gdspy.Text("FX22-W2-04-V3.2", 150, (800, -4350), layer=3)
#logo = gdspy.Text("Andersen Lab/QuTech", 150, (250, -4500), layer=3)
#main_cell.add(logo)
#main_cell.add(device_name)

#######################################################################################

##### N SHOULD BE EVEN!!!!!!!!!!
# N is a number of JJ in the array
N = 100

if int(N / 2) % 2 != 0:
    raise ValueError('N / 2 should be even!!!')

delta_x = 0.5
# delta_x_base = 1.5
delta_x_base = 1.5
# nanowire_length is  the length between nanowire's pads
nanowire_length = 130 + 3
real_nanowire_length = nanowire_length - delta_x * 2 - delta_x_base * 2

# very important shift at x direction
SHIFT = - nanowire_length / 2
nanowire_width = 1.3

delta = 0.410  # initial width of pad

nu_cell = 0

start_x = 0
start_y = 0

stop_x = start_x + nanowire_length
stop_y = start_y

# for vertical and horizontallines
step = 1.2
delta_W = (real_nanowire_length - 2 * step) / (N / 2 - 1)
delta_H = 3
print(delta_W)

corr1 = 1.5
corr2 = 1

u = 1 #undercut length

if delta == 0.410:
    cell = lib.new_cell('jj_array_and_JJ' + str(nu_cell))
    
    
    # left line
    
    left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x+SHIFT-1.6-5.5, start_y - delta / 2-3),
                                (start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y + delta / 2-3), layer=54)
    cell.add(left_line)
    
    undercut_left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y - delta / 2-3),
                                         (start_x + delta_x_base + delta_x + step + delta+corr2+0.7+SHIFT-4.0, start_y + delta / 2-3),
                                         layer=55)
    cell.add(undercut_left_line)
    
    
    
########################### ADDING JJ ARRAY ########################################################### 
    array_JJ = None
    k = 0 #  center
    p = 0 #  down 
    for i in range(int(N / 2)):
        if i<1:
            start_y=-3
            delta_H = 3.8
            start_x = -7
            corr2 = 8
        elif i<2:
            start_y = -3
            delta_H = 3
            start_x = 0
            corr2 = 1
        else:
            start_y=0
            delta_H = 3
            start_x = 0
            corr2 = 1
        if i<N//2-1:
            vertical_pad = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y - 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6),
                layer=54)
            vertical_circle_1 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y-5),
                                            radius=1.6*delta/2, layer=54, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_1, 'or', layer=54)
            vertical_circle_2 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y+5),
                                            radius=1.6*delta/2, layer=54, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_2, 'or', layer=54)


            undercut_vertical = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y + 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6 + u*2),

                layer=55)

            array_JJ = gdspy.boolean(array_JJ, vertical_pad, 'or')

            cell.add(vertical_pad)
            cell.add(undercut_vertical)
        if i==N//2-2:
            corr2 = 2.5
        if i % 2 != 0 and i<N//2-1: #and i != int(N / 2) - 1:
            
            if k % 2 == 0:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+ 3 ),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+ 3),
                layer=55)
                
            else:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+1 ),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+1 ),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2 +1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+1),
                layer=55)
                        
            k +=1
        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_center, 'or')

            cell.add(horizonatal_center)

            cell.add(undercut_horizontal_center)

        if i % 2 == 0 and i<N//2-1:
            
            if p % 2 == 0:
                
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta),
                layer=55)
                
            else:
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta + 1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H + 1.5),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H+1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta+1.5),
                layer=55)
                
            p += 1

        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_down, 'or')

            cell.add(horizonatal_down)

            cell.add(undercut_horizontal_center)
            
########################### ADDING JJ ###########################################################
            
deltaJJ = 0.120 # initial width of pad
pad_length_left = 2.4
pad_length_right = 1.7


        # Pads of junctions
left_padJJ = gdspy.Rectangle((-1.2, -26), (-1.2 + deltaJJ, -22), layer=54)
    
right_padJJ = gdspy.Rectangle((-3.0, -25.0), (1.2, -25.0 + deltaJJ), layer=54)
    
    
        # parts for undercuts
undercut_left_pad3 = gdspy.Rectangle((-1.2, -22), (-1.2 + deltaJJ, -21), layer=55)
    
undercut_right_pad4 = gdspy.Rectangle((1.2, -25.0), (2.2, -25.0 + deltaJJ), layer=55)
    


cell.add(left_padJJ)
cell.add(right_padJJ)
    
cell.add(undercut_left_pad3)
cell.add(undercut_right_pad4)
           



2.583673469387755


In [54]:
gdspy.LayoutViewer()

C:\Users\figenyilmaz\AppData\Local\Temp\ipykernel_20212\1312026100.py:1: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [55]:
lib.write_gds('Only-JJandArray_Dose_test_1by1mm.gds')

# Let's try the GDS file that we just created. We can only use Q1 and see. 
# Create geometry of a chip


In [72]:
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
import matplotlib.pyplot as plt

from qiskit_metal.toolbox_metal import math_and_overrides

from qiskit_metal.qlibrary.core import QComponent

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
# Each time you create a new quantum circuit design,
# you start by instantiating a QDesign class.

# The design class `DesignPlanar` is best for 2D circuit designs.
design = designs.DesignPlanar()
design.chips.main.material = 'silicon'
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '8.733 um'
design._chips['main']['size']['size_x'] = '1mm'
design._chips['main']['size']['size_y'] = '1mm'

# If you disable the next line with "overwrite_enabled", then you will need to
# delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True

gui = MetalGUI(design)

In [86]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

gds_cell_name_jj = 'junction_0'
gds_cell_name_inductor = 'jj_array0'

# Q1 details
options = dict(chip='main', nanowire = False, 
            orientation =90, pos_x = '0mm', pos_y = '0mm', pocket_width='900um',
            gds_cell_name=gds_cell_name_jj,
            gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = False,
                               ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = False, 
                           )
              )
               
q1 = FluxoniumPocket(design,'Q1', options = dict(
             **options))

gui.rebuild()

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  vertices = concatenate([

  vertices = concatenate([

  vertices = concatenate([



# Creating the GDS file

In [87]:
fluxonium_gds = design.renderers.gds
fluxonium_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': True,
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': 'Only-JJandArray_Dose_test_1by1mm.gds',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '200um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '300um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [88]:
# Choose file (with gradual junctions or note)
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions.gds'
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions_gradual.gds'
fluxonium_gds.options['path_filename'] = 'Only-JJandArray_Dose_test_1by1mm.gds'

In [89]:
fluxonium_gds.options['fabricate'] = True

In [90]:
# fluxonium_gds.export_to_gds('fluxonium_calibration_structure.gds')
fluxonium_gds.export_to_gds('JJDose-test.gds')

02:25PM 32s WARNING [_give_rotation_center_twopads]: In junction table, component=Q1 with name=inductor has width=0.001 smaller than cell dimension=0.017.
  all_polys = list(multi_poly)

  all_polys = list(multi_poly)



TypeError: 'NoneType' object is not iterable

In [8]:
gui.main_window.close()

True